# Aula 2: Pandas

- Pandas é um pacote Python que fornece estruturas de dados rápidas, flexíveis e expressivas, projetado para tornar o trabalho com dados "relacionais" ou "rotulados" fácil e intuitivo.

- Pandas funciona bem com os seguintes tipos de dados:
    - Dados tabulares com colunas de tipos heterogêneos
    - Dados de séries temporais ordenados e não ordenados
    - Dados de matriz arbitrários (tipificados homogeneamente ou heterogêneos) com rótulos de linha e coluna

- Instalação
    ```bash
    pip install pandas
    ```

- Documentação oficial: https://pandas.pydata.org/docs/

In [9]:
# Convenção de como importar o pandas
import pandas as pd

# Vamos também utilizar o numpy
import numpy as np

## Estruturas de dados fundamentais

O Pandas possui duas estruturas de dados principais

- `Series`: Array 1-D etiquetado com elementos de tipos arbitrários.

- `DataFrame`: Semelhando a um array 2-D ou uma tabela com linhas e colunas. Atua como um container de `Series`.

### `Series`

Podemos construir `Series` de algumas formas:

In [10]:
# Criando um `Series` sem explicitamente dando as etiquetas (o padrão é um `range`)
s = pd.Series([1, 2, 3, np.nan, 5])
print(s)

print("===")

# Exemplo usando etiquetas
s = pd.Series([1, 2, 3, np.nan, 5], index=["a", "b", "c", "d", "e"])
print(s)

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64
===
a    1.0
b    2.0
c    3.0
d    NaN
e    5.0
dtype: float64


`Series` se comporta como um `np.ndarray` e como um dicionária

In [11]:
s = pd.Series(np.arange(5), index=list("abcde"))

print(s)

print("======")

# Operações suportadas por um np.ndarray
print(s * 2)
print()
print(np.exp(s))

print("======")

# Operações suportadas por um dicionário
print("b" in s)
print(s["b"])
print(s.get("g", -1))

a    0
b    1
c    2
d    3
e    4
dtype: int64
a    0
b    2
c    4
d    6
e    8
dtype: int64

a     1.000000
b     2.718282
c     7.389056
d    20.085537
e    54.598150
dtype: float64
True
1
-1


A principal vantagem de um `Series` é que operações vetorizadas entre `Series` automaticamente alinham suas etiquetas 

In [12]:
s1 = pd.Series([1, 2, 3, 4], index=list("abcd"))
s2 = pd.Series([5, 6, 7, 8], index=list("bcde"))

s3 = s1 + s2
display(s1, s2, s3)

a    1
b    2
c    3
d    4
dtype: int64

b    5
c    6
d    7
e    8
dtype: int64

a     NaN
b     7.0
c     9.0
d    11.0
e     NaN
dtype: float64

### `DataFrame`

Criando um `DataFrame` passando os dados tabelados e as etiquetas das colunas e das linhas

In [13]:
df = pd.DataFrame(
    np.arange(3*4).reshape(3, 4),
    index=pd.date_range("2025-08-01", periods=3),
    columns=list("abcd"),
)
df

,a,b,c,d
2025-08-01,0,1,2,3
2025-08-02,4,5,6,7
2025-08-03,8,9,10,11


Criando um `DataFrame` com um dicionário

In [14]:
df = pd.DataFrame(
    {
        "A": 1.0, 
        "B": np.arange(4), 
        "C": pd.Categorical(["catA", "catB", "catC", "catD"]),
    }
)
display(df)

# Repare que as colunas podem possuir tipos diferentes
df.dtypes

,A,B,C
0,1.0,0,catA
1,1.0,1,catB
2,1.0,2,catC
3,1.0,3,catD


A     float64
B       int64
C    category
dtype: object

## Indexação

A indexação de um `DataFrame` pode ser feita utilizando as etiquetas das linhas e colunas, ou pela localização numérica das mesmas

In [15]:
df = pd.DataFrame(
    np.arange(3*4).reshape(3, 4),
    index=pd.date_range("2025-08-01", periods=3),
    columns=list("abcd"),
)
dates = df.index

df

,a,b,c,d
2025-08-01,0,1,2,3
2025-08-02,4,5,6,7
2025-08-03,8,9,10,11


Selecionando uma coluna (Repare que o resultado é um `Series`)

In [16]:
print(df["b"])
print("===")
print(type(df["b"]))

2025-08-01    1
2025-08-02    5
2025-08-03    9
Freq: D, Name: b, dtype: int64
===
<class 'pandas.core.series.Series'>


Selecionando uma linha

In [17]:
df.loc[dates[1]]

a    4
b    5
c    6
d    7
Name: 2025-08-02 00:00:00, dtype: int64

Podemos utilizar fatiamento e indices mais complicados. O exemplo a seguir seleciona todas
as linhas das colunas "a" e "b"

In [18]:
df.loc[:, ["a", "b"]]

,a,b
2025-08-01,0,1
2025-08-02,4,5
2025-08-03,8,9


Selecionando a primeira linha 

In [19]:
df.iloc[0]

a    0
b    1
c    2
d    3
Name: 2025-08-01 00:00:00, dtype: int64

Seleciona um único elemento

In [20]:
print(df.at[dates[2], "b"])

# ou utilizando a posição
print(df.iat[2, 1])

9
9


> OBS: Para acessar um elemento `df.loc[dates[2], "b"]` também funciona, mas o método `.at` é mais otimizado. O mesmo vale para fatiamento de linhas, `df[:2]` seleciona as duas primeira linhas, mas utilizar `.iloc` é mais rápido `df.iloc[:2]`. Em geral, prefira os métodos `.at`, `.iat`, `.loc` e `.iloc` para código mais otimizado.

Índices booleanos também funcionam. O exemplo a seguir seleciona todas as linhas em que a coluna "c" é maior do que 3

In [21]:
df[df["c"] > 3]

,a,b,c,d
2025-08-02,4,5,6,7
2025-08-03,8,9,10,11


É possível adicionar novas colunas em um `DataFrame` existente

In [22]:
df = pd.DataFrame(
    np.arange(3*4).reshape(3, 4),
    index=pd.date_range("2025-08-01", periods=3),
    columns=list("abcd"),
)

df["e"] = df["a"] * 2

df

,a,b,c,d,e
2025-08-01,0,1,2,3,0
2025-08-02,4,5,6,7,8
2025-08-03,8,9,10,11,16


## Trabalhando com dados faltantes

É muito comum termos dados com algumas entradas faltantes. Primeiro vamos ver como o `Numpy` lida com esses casos. 

### `Numpy` com dados faltantes/inválidos

O `Numpy` possui o módulo `np.ma` para lidar com arrays que podem possuir dados faltantes ou inválidos. Este módulo introduz o objeto `MaskedArray` que é uma junção entre um `ndarray` normal com uma máscara, que pode ser um `nomask` ou uma array de booleanos. Quando um elemento da máscara é `True`, o correspondente elemento do array é inválido.
Trabalhando com `MaskedArray`'s, os dados inválidos não são considerados nas computações

In [23]:
import numpy as np
from numpy import ma

x = np.array([1, 2, 3, -1, 5])

mask = x < 0
mx = ma.masked_array(x, mask=mask)

print(type(mx))
mx

<class 'numpy.ma.MaskedArray'>


masked_array(data=[1, 2, 3, --, 5],
             mask=[False, False, False,  True, False],
       fill_value=999999)

In [24]:
print(mx.mean())
print(mx.mean() == (1 + 2 + 3 + 5) / 4)

2.75
True


### `Pandas` com dados faltantes

O `Pandas` também funciona com dados faltantes.

In [65]:
df = pd.DataFrame(
    {
        "A": [1, 2, 3, None, 5],
        "B": [6, None, 8, None, 10],
    },
    dtype="Int64",
)
display(df)
df.info()

,A,B
0,1,6
1,2,<NA>
2,3,8
3,<NA>,<NA>
4,5,10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   A       4 non-null      Int64
 1   B       3 non-null      Int64
dtypes: Int64(2)
memory usage: 218.0 bytes


`NA` são desconsiderados em computações

In [71]:
df.mean()

A    2.75
B     8.0
dtype: Float64

`NA` são propagados em operações

In [73]:
df["A"] + df["B"]

0       7
1    <NA>
2      11
3    <NA>
4      15
dtype: Int64

Podemos verificar os `NA`'s estão e substituí-los

In [74]:
df.isna()

,A,B
0,False,False
1,False,True
2,False,False
3,True,True
4,False,False


In [80]:
df = pd.DataFrame(
    {
        "A": [1, 2, 3, None, 5],
        "B": [6, None, 8, None, 10],
    },
    dtype="Int64",
)
df = df.fillna(-1)
# df[df.isna()] = -1

df

,A,B
0,1,6
1,2,-1
2,3,8
3,-1,-1
4,5,10


## Salvando/Carregando `DataFrame`

In [83]:
df = pd.DataFrame(
    {
        "A": np.arange(5, 15),
        "B": np.arange(10) < 3,
        "C": list("abcdefghij"),
    }
)

df

,A,B,C
0,5,True,a
1,6,True,b
2,7,True,c
3,8,False,d
4,9,False,e
5,10,False,f
6,11,False,g
7,12,False,h
8,13,False,i
9,14,False,j


In [91]:
df.to_csv("my_dataframe.csv")

In [94]:
df_loaded = pd.read_csv("my_dataframe.csv", index_col=0)
df_loaded

,A,B,C
0,5,True,a
1,6,True,b
2,7,True,c
3,8,False,d
4,9,False,e
5,10,False,f
6,11,False,g
7,12,False,h
8,13,False,i
9,14,False,j
